## Summary of the project


<font color='#C6DEFF'><h2>1. Overview</h2></font> 


<font color='#98AFC7'><h3>Goal:</h3></font> 
The project is to built a ranking system for the merchants for BNPL company to partner up with.

<font color='#98AFC7'><h3>Metrics: </h3></font> 
1. Credibility - Whether the transaction is fraud

2. Predicted future Revenue 

3. Potential ability

4. Stability - monthly based variance

<font color='#C6DEFF'><h2>2. Preprocess</h2></font>

<font color='#98AFC7'><h3>External data set:</h3></font> 
Income and population information in different location divided by SA2 \

Feature we used: SA2(for merging), total earners, median age, median income, population density\


<font color='#F0FFF0'> <b>A demonstrate photo: </b></font> 

<img src="../plots/annual_income.png" alt="Geospatial of annual income" style="width:330px;height:300px;">

<font color='#98AFC7'><h3>Primary data set:</h3></font> 
The information for merchants, consumers and transaction among them. \

Feature we used: merhcant name, revenue level, take rate, category(preprocessed), gender, postcode, order time and dollor value

<font color='#FFE4B5'><h5>Category: </h5></font> 
The original dataset contain a feature called tag, it contains multiple meatures as a list, such as revenue level and products for the transaction. \
\
We seperated them into different features, and use the netral language process techniques to deal with the sentence that discribes the product (therefore implies the category). \
\
The stop words such as and, is are removed, the product behind the word "except" are removed. \
In the end five categories are obtained. \
They are: "outdoor", "art and gifts", "music and books", "home and technology" and "fashion and accessories".

<br>

<font color='#FFE4B5'><h5>Postcode: </h5></font> 
<font color='#FFDAB9'><b>Issue: </b></font> 
There are pobox mixed with postcode, which caused difficulty in merging external datasets. \

<b>Solution</b> \
Use the <a herf = https://auspost.com.au/postcode.>website </a> to find the corresponding postcode for each pobox. \
Some of the po box are not valid since 2006, they were treated as noise and droped

<font color='#98AFC7'><h3>Merging: </h3></font> 
<b>Join Key:</b> \
External: SA2 \
Internal: Postcode \
</br>

<font color='#FFDAB9'><b>Issue: </b></font> 
The SA2 and Postcode are two different unit for the location, they are not match. \
\
To achieve the most accurate match and lowest transaction lost, we overlayed the two keys one on top of the other to match their location. (shown in the graph, the red lines indicate postcode) \

\
Then the sitution of more than one postcodes match one SA2 and vice versa. \
We use the weighted (percentage of area of one key in the other key) average to calculate the value  of the feature values.

<img src="../plots/overlay.png" alt="overlay map" style="width:330px;height:300px;">

<font color='#C6DEFF'><h2>3. Genetrate Ranking Criteria</h2></font>

<font color='#98AFC7'><h3>1. Credibility: </h3></font> 
<b>Motivation: </b>  \
We were given the fraud probability for each consumer with daily guanularity, we want to utilize it as a ranking criteria. \
\
Moreover, there are many outliers, such as suddent jump in the dollar value of a particular day for one consumer. Instead of removing them, It is more useful to label them and includes in the ranking metric. \
\
The above two features were used to train a 3-mean clustering model. The result (as shown below) is very satisfiable,the two features are positively propotional to each other, the cluster with highest fraud probability also has the highed outlier rate. Then we labeled each group to high, median and low credibility to use as a factor for ranking system. \

<img src="../plots/fraud.png" alt="clustering result" style="width:330px;height:75px;">



<font color='#98AFC7'><h3>2. Revenue: </h3></font> 